We will dissect each part of the backtester to see how they work.
# 1. The DataHandler

In [14]:
from data_handler import HistoricalPolygonDataHandler
from polygon.tickers import get_id
from datetime import datetime, date
from event import MarketEvent
import queue
import pandas as pd
import numpy as np

In [3]:
events = queue.Queue()

In [4]:
data_handler = HistoricalPolygonDataHandler(events, start_date=date(2023, 8, 1), end_date=date(2023, 9, 1))
data_handler.load_data("AAPL", start_date=date(2023, 8, 1), end_date=date(2023, 9, 1))
data_handler.load_data("AA", start_date=date(2023, 7, 1), end_date=date(2023, 9, 1))

In [7]:
data_handler._all_bars["AAPL"]

,open,high,low,close,close_original,volume,tradeable,halted
datetime,,,,,,,,
2023-08-01 09:30:00,196.235,196.4900,195.930,196.0100,196.0100,898825,True,False
2023-08-01 09:31:00,196.050,196.2980,196.000,196.1100,196.1100,153387,True,False
2023-08-01 09:32:00,196.110,196.1400,195.800,196.0200,196.0200,162917,True,False
2023-08-01 09:33:00,196.050,196.1900,195.950,195.9658,195.9658,124321,True,False
2023-08-01 09:34:00,195.950,196.3961,195.925,196.3900,196.3900,161667,True,False
...,...,...,...,...,...,...,...,...
2023-09-01 15:55:00,189.380,189.4900,189.370,189.4850,189.4850,284447,True,False
2023-09-01 15:56:00,189.485,189.5100,189.450,189.4850,189.4850,250582,True,False
2023-09-01 15:57:00,189.485,189.5400,189.440,189.4650,189.4650,342040,True,False


In [7]:
data_handler.get_loaded_symbols()

['AAPL', 'AA']

In [8]:
for i in range(10):  
    data_handler.next()

TypeError: HistoricalPolygonDataHandler.next() takes 1 positional argument but 2 were given

In [ ]:
len(data_handler._latest_bars["AAPL"])

10

In [ ]:
data_handler.get_latest_bars("AAPL", N=2)

,open,high,low,close,close_original,volume,tradeable,halted
2023-08-01 09:38:00,196.3950,196.49,196.3871,196.43,196.43,128663,True,False
2023-08-01 09:39:00,196.4205,196.59,196.3700,196.55,196.55,235643,True,False


# 2. Broker

In [ ]:
from broker import SimulatedBroker
from event import OrderEvent
broker = SimulatedBroker(events, data_handler)
order = OrderEvent(datetime(2023, 8, 1, hour=9, minute=39), "AAPL", side="BUY", quantity=10)

2023-08-01T09:39:00 | ORDER BUY 10 of AAPL


In [ ]:
broker.execute_order(order)

In [ ]:
for i in range(10):
    events.get() # Remove the MarketEvents
event = events.get()
event

In [ ]:
event.total_fill

1965.5

# 3. Portfolio

In [ ]:
from portfolio import StandardPortfolio
from event import FillEvent
from data_handler import HistoricalPolygonDataHandler
from polygon.tickers import get_id
from datetime import datetime, date
import queue
events = queue.Queue()

In [ ]:
data_handler = HistoricalPolygonDataHandler(events, start_date=datetime(2023, 8, 1), end_date=date(2023, 9, 1))
data_handler.load_data("AAPL", start=date(2023, 8, 1), end=date(2023, 9, 1))
data_handler.load_data("AA", start=date(2023, 7, 1), end=date(2023, 9, 1))

portfolio = StandardPortfolio(events, data_handler, start_date=datetime(2023, 8, 1, hour=9, minute=30))

In [ ]:
portfolio.current_equity

10000.0

In [ ]:
portfolio.current_positions_value

0

In [ ]:
for i in range(1, 5):  
    data_handler.next(datetime(2023, 8, 1, hour=9, minute=30+i))
    portfolio.append_portfolio_log(dt=datetime(2023, 8, 1, hour=9, minute=30+i))
data_handler.get_latest_bars("AAPL", N=2)

,open,high,low,close,close_original,volume,tradeable,halted
2023-08-01 09:33:00,196.05,196.1900,195.950,195.9658,195.9658,124321,True,False
2023-08-01 09:34:00,195.95,196.3961,195.925,196.3900,196.3900,161667,True,False


In [ ]:
fill = FillEvent(dt=datetime(2023, 8, 1, hour=9, minute=34), symbol="AAPL", side='BUY', quantity=25, fill_price=196.39, fees=100)
portfolio.update_from_fill(fill)

In [ ]:
fill = FillEvent(dt=datetime(2023, 8, 1, hour=9, minute=34), symbol="AAPL", side='BUY', quantity=25, fill_price=196.39, fees=100)
portfolio.update_from_fill(fill)

In [ ]:
portfolio.current_positions

{'AAPL': 50}

In [ ]:
portfolio.current_cash

-19.5

In [ ]:
portfolio.current_positions_value

9819.5

In [ ]:
portfolio.current_equity

9800.0

In [ ]:
for i in range(5):  
    data_handler.next(datetime(2023, 8, 1, hour=9, minute=35+i))
    portfolio.append_portfolio_log(dt=datetime(2023, 8, 1, hour=9, minute=35+i))
data_handler.get_latest_bars("AAPL", N=2)

,open,high,low,close,close_original,volume,tradeable,halted
2023-08-01 09:38:00,196.3950,196.49,196.3871,196.43,196.43,128663,True,False
2023-08-01 09:39:00,196.4205,196.59,196.3700,196.55,196.55,235643,True,False


In [ ]:
fill = FillEvent(dt=datetime(2023, 8, 1, hour=9, minute=39), symbol="AAPL", side='SELL', quantity=60, fill_price=196.55, fees=100)
portfolio.update_from_fill(fill)

In [ ]:
for i in range(5):  
    data_handler.next(datetime(2023, 8, 1, hour=9, minute=40+i))
    portfolio.append_portfolio_log(dt=datetime(2023, 8, 1, hour=9, minute=40+i))
data_handler.get_latest_bars("AAPL", N=2)

,open,high,low,close,close_original,volume,tradeable,halted
2023-08-01 09:43:00,196.4814,196.62,196.45,196.5350,196.5350,93577,True,False
2023-08-01 09:44:00,196.5300,196.56,196.42,196.4798,196.4798,83679,True,False


In [ ]:
data_handler.next(datetime(2023, 8, 1, hour=9, minute=45))
fill = FillEvent(dt=datetime(2023, 8, 1, hour=9, minute=45), symbol="AAPL", side='BUY', quantity=10, fill_price=196.50, fees=100)
portfolio.update_from_fill(fill)
portfolio.append_portfolio_log(dt=datetime(2023, 8, 1, hour=9, minute=45))

In [ ]:
# I prefer only logging the performance each day instead of every bar, else this list gets unnecessarily long
portfolio.create_df_from_holdings_log()

,equity,cash,positions_value,positions,return,return_cum
datetime,,,,,,
2023-08-01 09:30:00,10000.000,10000.0,0.000,{},0.000000,0.000000
2023-08-01 09:31:00,10000.000,10000.0,0.000,{},0.000000,0.000000
2023-08-01 09:32:00,10000.000,10000.0,0.000,{},0.000000,0.000000
2023-08-01 09:33:00,10000.000,10000.0,0.000,{},0.000000,0.000000
2023-08-01 09:34:00,10000.000,10000.0,0.000,{},0.000000,0.000000
2023-08-01 09:35:00,9792.500,-19.5,9812.000,{'AAPL': 50},-0.020750,-0.020750
2023-08-01 09:36:00,9799.500,-19.5,9819.000,{'AAPL': 50},0.000715,-0.020050
2023-08-01 09:37:00,9800.500,-19.5,9820.000,{'AAPL': 50},0.000102,-0.019950
2023-08-01 09:38:00,9802.000,-19.5,9821.500,{'AAPL': 50},0.000153,-0.019800


In [ ]:
fills_log = pd.DataFrame(portfolio.fills_log)
fills_log.set_index('datetime', inplace=True)
fills_log

,symbol,side,quantity,fill_price,fees
datetime,,,,,
2023-08-01 09:34:00,AAPL,BUY,25,196.39,100
2023-08-01 09:34:00,AAPL,BUY,25,196.39,100
2023-08-01 09:39:00,AAPL,SELL,60,196.55,100
2023-08-01 09:45:00,AAPL,BUY,10,196.50,100


# 4. Portfolio - Trade log
The trade log is the list of profit/losses from every trade. All information we need to calculate that are in the fills log. However this is not straightforward. If you look at the fills log above, what are the trades? Should the BUY 25 from the first 2 trades be grouped? If the short trade was 40 shares instead, how would be assign them to the first 2 trades? Do we see that as closing the the first and partially closing the second? What about positions that are still open?

When constructing the trade log, we use the following rules:
1. We never group opening trades. So we see the first 2 trades as seperate trades.
2. Trades in the opposing direction are assigned as FIFO. If the opposing direction is larger than the entire posiiton, we see this as a new position. So the SELL -60 means we exit the 1st and 2nd trade, and create a new trade that is short 10 shares. If the trade was SELL -40 instead, we would assign -25 to the first trade and -15 to the second trade. Because we use FIFO.
3. For open positions, we can calculate the current P/L.

(All P/L are realized. I will not bother with unrealized P/L in the trade log.)

In [ ]:
def fills_to_trades(fills_log):
    trade_log = pd.DataFrame(columns=['datetime_in', 'symbol', 'side', 'quantity', 'entry', 'exit', 'datetime_out', 'fees', 'net P/L %', 'net P/L $', 'remaining_qty'])
    for dt, trade in fills_log.iterrows():
        symbol = trade['symbol']
        side = trade['side']
        opposite_side = 'SELL' if side == 'BUY' else 'BUY'
        quantity = trade['quantity']
        fill_price = trade['fill_price']
        fees = trade['fees']

        current_position_in_symbol_opposite = trade_log[(trade_log['symbol'] == symbol) & (trade_log['side'] == opposite_side) & (trade_log['remaining_qty'] > 0)]
        if len(current_position_in_symbol_opposite) == 0:
            # If no open trades in this symbol in the opposite direction, create new trade
            trade_log.loc[len(trade_log)] = [dt, symbol, side, quantity, fill_price, np.nan, np.nan, fees, np.nan, np.nan, quantity]
        else:
            # Else we (partially) close the trade(s) and create a new trade if a net position remains. Using FIFO.
            for index, open_trade in current_position_in_symbol_opposite.iterrows():
                remaining_qty_open_trade = open_trade['remaining_qty']
                already_filled_qty_open_trade = open_trade['quantity'] - open_trade['remaining_qty']
                current_average_fill = open_trade['exit']

                # Partial close of open_trade
                if quantity < remaining_qty_open_trade:
                    if np.isnan(current_average_fill):
                        trade_log.loc[index, "exit"] = fill_price
                    else:
                        average_fill_exit = ((current_average_fill * already_filled_qty_open_trade) + (fill_price * quantity))/(already_filled_qty_open_trade + quantity) # Calculate new average fill
                        trade_log.loc[index, "exit"] = average_fill_exit

                    trade_log.loc[index, "remaining_qty"] -= quantity
                    trade_log.loc[index, "fees"] += fees
                    break # We don't have to look at the next trade

                # Full close of open_trade
                elif quantity >= remaining_qty_open_trade:
                    if np.isnan(current_average_fill):
                        trade_log.loc[index, "exit"] = fill_price
                    else:
                        average_fill_exit = ((current_average_fill * already_filled_qty_open_trade) + (fill_price * quantity))/(already_filled_qty_open_trade + quantity) # Calculate new average fill
                        trade_log.loc[index, "exit"] = average_fill_exit

                    trade_log.loc[index, "remaining_qty"] = 0
                    trade_log.loc[index, "fees"] += fees
                    trade_log.loc[index, "datetime_out"] = dt

                    if quantity == remaining_qty_open_trade:
                        break # We don't have to look at the next trade
                    else:
                        quantity = quantity - remaining_qty_open_trade # Calculate remaining quantity

                        # If we are at the end and there is still a remaining quantity, that is a new position
                        if index == len(current_position_in_symbol_opposite)-1:
                            trade_log.loc[len(trade_log)] = [dt, symbol, side, quantity, fill_price, np.nan, np.nan, fees, np.nan, np.nan, quantity]
    trade_log['datetime_out'] = pd.to_datetime(trade_log['datetime_out'])
    return trade_log
            

In [ ]:
def calculate_PNL_trade_log(trade_log):
    trade_log["direction"] = np.where(trade_log["side"] == "BUY", 1, -1)
    trade_log["filled_qty"] = trade_log["quantity"] - trade_log["remaining_qty"]

    trade_log["net P/L $"] = (
        trade_log["filled_qty"] * trade_log["direction"]
    ) * (trade_log["exit"] - trade_log["entry"]) - trade_log["fees"]
    trade_log["net P/L %"] = 100 * trade_log["net P/L $"] / (trade_log["entry"] * trade_log['quantity'])

    trade_log["net P/L $"] = trade_log["net P/L $"].round(2)
    trade_log["net P/L %"] = trade_log["net P/L %"].round(2)
    return trade_log.drop(columns=["direction", "filled_qty"])

Using the above example:

In [ ]:
fills_log

,symbol,side,quantity,fill_price,fees
datetime,,,,,
2023-08-01 09:34:00,AAPL,BUY,25,196.39,100
2023-08-01 09:34:00,AAPL,BUY,25,196.39,100
2023-08-01 09:39:00,AAPL,SELL,60,196.55,100
2023-08-01 09:45:00,AAPL,BUY,10,196.50,100


In [ ]:
calculate_PNL_trade_log(fills_to_trades(fills_log))

,datetime_in,symbol,side,quantity,entry,exit,datetime_out,fees,net P/L %,net P/L $,remaining_qty
0,2023-08-01 09:34:00,AAPL,BUY,25,196.39,196.55,2023-08-01 09:39:00,200,-3.99,-196.0,0
1,2023-08-01 09:34:00,AAPL,BUY,25,196.39,196.55,2023-08-01 09:39:00,200,-3.99,-196.0,0
2,2023-08-01 09:39:00,AAPL,SELL,10,196.55,196.50,2023-08-01 09:45:00,200,-10.15,-199.5,0


Closing order bigger than opening order:

In [ ]:
fills_log = pd.DataFrame([[datetime(2023, 8, 1, hour=10, minute=0), "AAPL", "BUY", 100, 10, 1], 
                          [datetime(2023, 8, 1, hour=10, minute=1), "AAPL", "SELL", 200, 15, 1],
                          ], columns=['datetime', 'symbol', 'side', 'quantity', 'fill_price', 'fees'])
fills_log.set_index('datetime', inplace=True)
fills_log

,symbol,side,quantity,fill_price,fees
datetime,,,,,
2023-08-01 10:00:00,AAPL,BUY,100,10,1
2023-08-01 10:01:00,AAPL,SELL,200,15,1


In [ ]:
calculate_PNL_trade_log(fills_to_trades(fills_log))

,datetime_in,symbol,side,quantity,entry,exit,datetime_out,fees,net P/L %,net P/L $,remaining_qty
0,2023-08-01 10:00:00,AAPL,BUY,100,10,15.0,2023-08-01 10:01:00,2,49.8,498.0,0
1,2023-08-01 10:01:00,AAPL,SELL,100,15,NaN,NaT,1,NaN,NaN,100


Closing order smaller than opening order:

In [ ]:
fills_log = pd.DataFrame([[datetime(2023, 8, 1, hour=10, minute=0), "AAPL", "BUY", 100, 10, 1], 
                          [datetime(2023, 8, 1, hour=10, minute=1), "AAPL", "SELL", 50, 15, 1],
                          ], columns=['datetime', 'symbol', 'side', 'quantity', 'fill_price', 'fees'])
fills_log.set_index('datetime', inplace=True)
fills_log

,symbol,side,quantity,fill_price,fees
datetime,,,,,
2023-08-01 10:00:00,AAPL,BUY,100,10,1
2023-08-01 10:01:00,AAPL,SELL,50,15,1


Two fully closed orders:

In [ ]:
calculate_PNL_trade_log(fills_to_trades(fills_log))

,datetime_in,symbol,side,quantity,entry,exit,datetime_out,fees,net P/L %,net P/L $,remaining_qty
0,2023-08-01 10:00:00,AAPL,BUY,100,10,15.0,NaT,2,24.8,248.0,50


In [ ]:
fills_log = pd.DataFrame([[datetime(2023, 8, 1, hour=10, minute=0), "AAPL", "BUY", 100, 10, 100], 
                          [datetime(2023, 8, 5, hour=11, minute=0), "AAPL", "SELL", 100, 15, 100],
                          [datetime(2023, 8, 6, hour=12, minute=0), "AAPL", "SELL", 100, 20, 100],
                          [datetime(2023, 8, 15, hour=13, minute=0), "AAPL", "BUY", 100, 15, 100],
                          [datetime(2023, 9, 6, hour=12, minute=0), "TSLA", "BUY", 10, 200, 100],
                          [datetime(2023, 9, 15, hour=13, minute=0), "TSLA", "SELL", 10, 100, 100],
                          ], columns=['datetime', 'symbol', 'side', 'quantity', 'fill_price', 'fees'])
fills_log.set_index('datetime', inplace=True)
fills_log

,symbol,side,quantity,fill_price,fees
datetime,,,,,
2023-08-01 10:00:00,AAPL,BUY,100,10,100
2023-08-05 11:00:00,AAPL,SELL,100,15,100
2023-08-06 12:00:00,AAPL,SELL,100,20,100
2023-08-15 13:00:00,AAPL,BUY,100,15,100
2023-09-06 12:00:00,TSLA,BUY,10,200,100
2023-09-15 13:00:00,TSLA,SELL,10,100,100


In [ ]:
trade_log = calculate_PNL_trade_log(fills_to_trades(fills_log))
trade_log

,datetime_in,symbol,side,quantity,entry,exit,datetime_out,fees,net P/L %,net P/L $,remaining_qty
0,2023-08-01 10:00:00,AAPL,BUY,100,10,15.0,2023-08-05 11:00:00,200,30.0,300.0,0
1,2023-08-06 12:00:00,AAPL,SELL,100,20,15.0,2023-08-15 13:00:00,200,15.0,300.0,0
2,2023-09-06 12:00:00,TSLA,BUY,10,200,100.0,2023-09-15 13:00:00,200,-60.0,-1200.0,0


# 5. Statistics

In [ ]:
portfolio_log = pd.DataFrame(
    [
        [datetime(2023, 8, 1, hour=10, minute=0), 10000, 10000, 0, {}],
        [datetime(2023, 8, 2, hour=11, minute=0), 9000, 5000, 4000, {}],
        [datetime(2023, 8, 3, hour=12, minute=0), 12000, 5000, 7000, {}],
        [datetime(2024, 8, 4, hour=13, minute=0), 13000, 13000, 0, {}],
    ],
    columns=["datetime", "equity", "cash", "positions_value", "positions"],
)
portfolio_log.set_index("datetime", inplace=True)
portfolio_log["return"] = portfolio_log["equity"].pct_change()
portfolio_log["return_cum"] = (1.0 + portfolio_log["return"]).cumprod() - 1
portfolio_log = portfolio_log.fillna(value=0)

In [ ]:
portfolio_log

,equity,cash,positions_value,positions,return,return_cum
datetime,,,,,,
2023-08-01 10:00:00,10000,10000,0,{},0.000000,0.0
2023-08-02 11:00:00,9000,5000,4000,{},-0.100000,-0.1
2023-08-03 12:00:00,12000,5000,7000,{},0.333333,0.2
2024-08-04 13:00:00,13000,13000,0,{},0.083333,0.3


In [ ]:
import performance
# Statistics are all over the place because of the very short portfolio/trade logs.
statistics = {'Annual return %': performance.calculate_annual_return(portfolio_log),
              'Sharpe': performance.calculate_sharpe(portfolio_log),
              'Sortina': performance.calculate_sortina(portfolio_log),
              'Winning months %': performance.calculate_winning_months(portfolio_log),
              'Time in market %': performance.calculate_time_in_market(portfolio_log),
              'Average profit %': performance.calculate_average_profit(trade_log),
              'Average duration per trade': f'{performance.calculate_average_trade_duration(trade_log)[0]}d{performance.calculate_average_trade_duration(trade_log)[1]}h{performance.calculate_average_trade_duration(trade_log)[2]}m',
              'Profit factor': performance.calculate_profit_factor(trade_log),
              'Trades/month': performance.calculate_trades_per_month(portfolio_log, trade_log)}
statistics

{'Annual return %': -69.6,
 'Sharpe': 6.78,
 'Sortina': nan,
 'Winning months %': 15.0,
 'Time in market %': 0.01,
 'Average profit %': -5.0,
 'Average duration per trade': '7d9h0m',
 'Profit factor': 0.38,
 'Trades/month': 0.2}

In [1]:
from polygon.times import get_market_calendar, get_market_minutes, get_market_dates
from datetime import date, time
import pandas as pd
calendar = get_market_calendar("datetime")
minutes = get_market_minutes(start_date=date(2023, 8, 1), end_date=date(2023, 9, 1), extended_hours=False)

The clock

In [ ]:
def create_clock(start_date, end_date, extended_hours):
    """A generator that yields the datetime. This simulates a clock.

    Args:
        start_date (Date): the start date
        end_date (Date): the end date
        extended_hours (bool): whether to include extended hours
    """
    for dt in get_market_minutes(start_date, end_date, extended_hours):
        yield dt

In [ ]:
clock = create_clock(minutes)

In [ ]:
for i in range(10):
    print(next(clock))

# 6. Calendar


In [2]:
from polygon.times import get_market_calendar, get_market_minutes, get_market_dates
from datetime import date
market_hours = get_market_calendar(format="time")
market_hours.head(3)

,premarket_open,regular_open,regular_close,postmarket_close
2019-01-02,04:00:00,09:30:00,15:59:00,19:59:00
2019-01-03,04:00:00,09:30:00,15:59:00,19:59:00
2019-01-04,04:00:00,09:30:00,15:59:00,19:59:00


In [5]:
market_hours = get_market_calendar(format="datetime", timeframe=5)
market_hours.head(3)

,premarket_open,regular_open,regular_close,postmarket_close
2019-01-02,2019-01-02 04:00:00,2019-01-02 09:30:00,2019-01-02 15:55:00,2019-01-02 19:55:00
2019-01-03,2019-01-03 04:00:00,2019-01-03 09:30:00,2019-01-03 15:55:00,2019-01-03 19:55:00
2019-01-04,2019-01-04 04:00:00,2019-01-04 09:30:00,2019-01-04 15:55:00,2019-01-04 19:55:00


TO DO: 
* Make it work for m5 and d1
* Make GAT orders. All orders should be executed at the next open. This avoid trading in halts for minute data and avoiding look-aheas bias for daily timeframe strategies.
* Fix this notebook

In [4]:
from polygon.data import get_data
data = get_data("AAPL", start_date = date(2023, 8, 1), end_date = date(2023, 9, 1))

In [5]:
data.index

DatetimeIndex(['2023-08-01', '2023-08-02', '2023-08-03', '2023-08-04',
               '2023-08-07', '2023-08-08', '2023-08-09', '2023-08-10',
               '2023-08-11', '2023-08-14', '2023-08-15', '2023-08-16',
               '2023-08-17', '2023-08-18', '2023-08-21', '2023-08-22',
               '2023-08-23', '2023-08-24', '2023-08-25', '2023-08-28',
               '2023-08-29', '2023-08-30', '2023-08-31', '2023-09-01'],
              dtype='datetime64[ns]', name='datetime', freq=None)